# 使用 MyScale 的向量 SQL 检索器

>[MyScale](https://docs.myscale.com/en/) 是一个集成的向量数据库。您可以通过 SQL 访问您的数据库，也可以通过 LangChain 访问。MyScale 可以利用[各种数据类型和过滤函数](https://blog.myscale.com/2023/06/06/why-integrated-database-solution-can-boost-your-llm-apps/#filter-on-anything-without-constraints)。无论您是扩展数据还是将系统扩展到更广泛的应用程序，它都将提升您的 LLM 应用程序。

In [ ]:
# 安装所需的Python包
!pip3 install clickhouse-sqlalchemy InstructorEmbedding sentence_transformers openai langchain-experimental

In [ ]:
import getpass
from os import environ

# 导入所需的模块和类
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql.vector_sql import VectorSQLDatabaseChain
from langchain_openai import OpenAI
from sqlalchemy import MetaData, create_engine

# 设置连接信息
MYSCALE_HOST = "msc-4a9e710a.us-east-1.aws.staging.myscale.cloud"
MYSCALE_PORT = 443
MYSCALE_USER = "chatdata"
MYSCALE_PASSWORD = "myscale_rocks"
OPENAI_API_KEY = getpass.getpass("OpenAI API Key:")

# 创建 ClickHouse 引擎
engine = create_engine(
    f"clickhouse://{MYSCALE_USER}:{MYSCALE_PASSWORD}@{MYSCALE_HOST}:{MYSCALE_PORT}/default?protocol=https"
)

# 创建元数据对象
metadata = MetaData(bind=engine)

# 设置 OpenAI API 密钥
environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [ ]:
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_experimental.sql.vector_sql import VectorSQLOutputParser

output_parser = VectorSQLOutputParser.from_embeddings(
    model=HuggingFaceInstructEmbeddings(
        model_name="hkunlp/instructor-xl", model_kwargs={"device": "cpu"}
    )
)

In [ ]:
# 导入必要的模块
from langchain.callbacks import StdOutCallbackHandler
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_experimental.sql.prompt import MYSCALE_PROMPT
from langchain_experimental.sql.vector_sql import VectorSQLDatabaseChain
from langchain_openai import OpenAI

# 创建一个VectorSQLDatabaseChain对象
chain = VectorSQLDatabaseChain(
    llm_chain=LLMChain(
        llm=OpenAI(openai_api_key=OPENAI_API_KEY, temperature=0),  # 使用OpenAI的API密钥和温度参数初始化OpenAI对象
        prompt=MYSCALE_PROMPT,  # 设置prompt为MYSCALE_PROMPT
    ),
    top_k=10,  # 设置top_k参数为10
    return_direct=True,  # 设置return_direct参数为True
    sql_cmd_parser=output_parser,  # 使用output_parser解析SQL命令
    database=SQLDatabase(engine, None, metadata),  # 使用engine和metadata初始化SQLDatabase对象
)

# 导入pandas模块
import pandas as pd

# 运行chain对象，并将结果转换为DataFrame格式
pd.DataFrame(
    chain.run(
        "Please give me 10 papers to ask what is PageRank?",  # 输入查询语句
        callbacks=[StdOutCallbackHandler()],  # 设置回调函数为StdOutCallbackHandler
    )
)

## SQL数据库作为检索器

In [ ]:
from langchain.chains.qa_with_sources.retrieval import RetrievalQAWithSourcesChain
from langchain_experimental.retrievers.vector_sql_database import (
    VectorSQLDatabaseChainRetriever,
)
from langchain_experimental.sql.prompt import MYSCALE_PROMPT
from langchain_experimental.sql.vector_sql import (
    VectorSQLDatabaseChain,
    VectorSQLRetrieveAllOutputParser,
)
from langchain_openai import ChatOpenAI

output_parser_retrieve_all = VectorSQLRetrieveAllOutputParser.from_embeddings(
    output_parser.model
)  # 从嵌入中创建VectorSQLRetrieveAllOutputParser对象

chain = VectorSQLDatabaseChain.from_llm(
    llm=OpenAI(openai_api_key=OPENAI_API_KEY, temperature=0),  # 使用OpenAI模型创建llm对象
    prompt=MYSCALE_PROMPT,  # 使用MYSCALE_PROMPT作为提示
    top_k=10,  # 返回前10个结果
    return_direct=True,  # 返回直接结果
    db=SQLDatabase(engine, None, metadata),  # 使用给定的engine、None和metadata创建SQLDatabase对象
    sql_cmd_parser=output_parser_retrieve_all,  # 使用output_parser_retrieve_all作为sql_cmd_parser
    native_format=True,  # 使用原生格式
)

# 您需要所有这些键来获取文档
retriever = VectorSQLDatabaseChainRetriever(
    sql_db_chain=chain, page_content_key="abstract"  # 使用chain和"abstract"作为page_content_key创建VectorSQLDatabaseChainRetriever对象
)

document_with_metadata_prompt = PromptTemplate(
    input_variables=["page_content", "id", "title", "authors", "pubdate", "categories"],  # 输入变量列表
    template="Content:\n\tTitle: {title}\n\tAbstract: {page_content}\n\tAuthors: {authors}\n\tDate of Publication: {pubdate}\n\tCategories: {categories}\nSOURCE: {id}",  # 模板字符串
)

chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(
        model_name="gpt-3.5-turbo-16k", openai_api_key=OPENAI_API_KEY, temperature=0.6  # 使用给定的模型名称、OPENAI_API_KEY和温度创建ChatOpenAI对象
    ),
    retriever=retriever,  # 使用retriever作为retriever参数
    chain_type="stuff",  # 使用"stuff"作为chain_type
    chain_type_kwargs={
        "document_prompt": document_with_metadata_prompt,  # 使用document_with_metadata_prompt作为document_prompt参数
    },
    return_source_documents=True,  # 返回源文档
)
ans = chain(
    "Please give me 10 papers to ask what is PageRank?",  # 输入问题
    callbacks=[StdOutCallbackHandler()],  # 使用StdOutCallbackHandler作为回调函数
)
print(ans["answer"])  # 打印答案